# 如何给agent正确的增加记忆
- 将Memory插入到提示词模板中

In [4]:
from langchain.agents import ( 
    load_tools,
    AgentType,
    initialize_agent,
)
from langchain.agents import Tool
from langchain.agents import AgentType
from langchain.memory import ConversationBufferMemory
from langchain_openai  import ChatOpenAI
from langchain.utilities import SerpAPIWrapper
from langchain.agents import initialize_agent
from langchain.chains import LLMMathChain
from langchain.prompts import  MessagesPlaceholder
from dotenv import load_dotenv
# Load environment variables from openai.env file
load_dotenv()

True

In [31]:
llm=ChatOpenAI(
    temperature=0,
    model="gpt-3.5-turbo"
)

### 构建agent可用工具

In [7]:
#加载工具
tools = load_tools(["serpapi", "llm-math"], llm=llm),

In [6]:
search = SerpAPIWrapper()
#创建一个数学计算工具
llm_math_chain = LLMMathChain(
    llm=llm,
    verbose=True
)

tools = [
    Tool(
        name = "Search",
        func=search.run,
        description="useful for when you need to answer questions about current events or the current state of the world"
    ),
    Tool(
        name="Calculator",
        func=llm_math_chain.run,
        description="useful for when you need to answer questions about math"
    ),
]
print(tools)

[Tool(name='Search', description='useful for when you need to answer questions about current events or the current state of the world', func=<bound method SerpAPIWrapper.run of SerpAPIWrapper(search_engine=<class 'serpapi.google_search.GoogleSearch'>, params={'engine': 'google', 'google_domain': 'google.com', 'gl': 'us', 'hl': 'en'}, serpapi_api_key='4ba6f3d3a1010b2252ca53ebe8fc77c1727eadbc894149a03a5fb5039d86194c', aiosession=None)>), Tool(name='Calculator', description='useful for when you need to answer questions about math', func=<bound method Chain.run of LLMMathChain(verbose=True, llm_chain=LLMChain(prompt=PromptTemplate(input_variables=['question'], template='Translate a math problem into a expression that can be executed using Python\'s numexpr library. Use the output of running this code to answer the question.\n\nQuestion: ${{Question with math problem.}}\n```text\n${{single line mathematical expression that solves the problem}}\n```\n...numexpr.evaluate(text)...\n```output\n

c:\Programs\Python\Python312\Lib\site-packages\langchain\chains\llm_math\base.py:57: UserWarning: Directly instantiating an LLMMathChain with an llm is deprecated. Please instantiate with llm_chain argument or using the from_llm class method.
  warnings.warn(


In [29]:
#添加memory记忆组件

memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

In [ ]:
# 定义agent
agent_chain = initialize_agent(
    tools, 
    llm, 
    agent=AgentType.OPENAI_FUNCTIONS, 
    verbose=True, 
    handle_parsing_errors=True,#处理解析错误
    memory=memory #记忆组件
)

In [ ]:
print(agent_chain.run("What is 1+1?"))

In [ ]:
print(agent_chain.agent.prompt.messages)
print(agent_chain.agent.prompt.messages[0])
print(agent_chain.agent.prompt.messages[1])
print(agent_chain.agent.prompt.messages[2])

In [ ]:
agent_chain = initialize_agent(
    tools, 
    llm, 
    agent=AgentType.OPENAI_FUNCTIONS, 
    verbose=True, 
    handle_parsing_errors=True,#处理解析错误
    agent_kwargs={
        "extra_prompt_messages":[MessagesPlaceholder(variable_name="chat_history"),MessagesPlaceholder(variable_name="agent_scratchpad")],
    },
    memory=memory #记忆组件
    )

In [ ]:
print(agent_chain.agent.prompt.messages)
print(agent_chain.agent.prompt.messages[0])
print(agent_chain.agent.prompt.messages[1])
print(agent_chain.agent.prompt.messages[2])

In [ ]:
agent_chain.run("好厉害，刚才我们都聊了什么？")